Bulk Write 작업

이 가이드에서는 PyMongo의 대량 쓰기 작업 기능을 활용하는 방법을 설명합니다. 쓰기 작업을 일괄 적으로 실행하면 네트워크 왕복 횟수가 줄어들고 쓰기 처리량이 증가합니다.

Bulk Insert

버전 2.6의 새로운 기능.

목록을 insert_many()메소드에 전달하여 document 배치를 삽입 할 수 있습니다. PyMongo는 MongoDB에서 허용하는 최대 메시지 크기를 기반으로 배치를 더 작은 하위 배치로 자동 분할하여 매우 큰 대량 삽입 작업을 지원합니다.

In [1]:
from pymongo import MongoClient
client = MongoClient()

In [5]:
db = client.bulk_example
db.bulk.insert_many(
    [{'i':i} for i in range(10000)]).inserted_ids

[ObjectId('5fcee5583090e67da89ca17e'),
 ObjectId('5fcee5583090e67da89ca17f'),
 ObjectId('5fcee5583090e67da89ca180'),
 ObjectId('5fcee5583090e67da89ca181'),
 ObjectId('5fcee5583090e67da89ca182'),
 ObjectId('5fcee5583090e67da89ca183'),
 ObjectId('5fcee5583090e67da89ca184'),
 ObjectId('5fcee5583090e67da89ca185'),
 ObjectId('5fcee5583090e67da89ca186'),
 ObjectId('5fcee5583090e67da89ca187'),
 ObjectId('5fcee5583090e67da89ca188'),
 ObjectId('5fcee5583090e67da89ca189'),
 ObjectId('5fcee5583090e67da89ca18a'),
 ObjectId('5fcee5583090e67da89ca18b'),
 ObjectId('5fcee5583090e67da89ca18c'),
 ObjectId('5fcee5583090e67da89ca18d'),
 ObjectId('5fcee5583090e67da89ca18e'),
 ObjectId('5fcee5583090e67da89ca18f'),
 ObjectId('5fcee5583090e67da89ca190'),
 ObjectId('5fcee5583090e67da89ca191'),
 ObjectId('5fcee5583090e67da89ca192'),
 ObjectId('5fcee5583090e67da89ca193'),
 ObjectId('5fcee5583090e67da89ca194'),
 ObjectId('5fcee5583090e67da89ca195'),
 ObjectId('5fcee5583090e67da89ca196'),
 ObjectId('5fcee5583090e6

In [7]:
db.bulk.count_documents({})

10000

Mixed Bulk Write 작업

버전 2.7의 새로운 기능.

PyMongo는 혼합 대량 쓰기 작업 실행도 지원합니다. 대량 쓰기 작업 API를 사용하여 삽입, 업데이트 및 제거 작업의 일괄 처리를 함께 실행할 수 있습니다.

Ordered Bulk Write 작업

정렬된 대량 쓰기 작업은 일괄 처리되어 직렬 실행을 위해 제공된 순서대로 서버로 전송됩니다. 반환 값은 수행 된 작업의 유형과 개수를 설명하는 BulkWriteResult의 인스턴스입니다.

In [12]:
from pprint import pprint
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
result = db.bulk.bulk_write([
    DeleteMany({}), # Remove all documents from the previous example.
    InsertOne({'_id': 1}),
    InsertOne({'_id': 2}),
    InsertOne({'_id': 3}),
    UpdateOne({'_id': 1}, {'$set': {'foo': 'bar'}}),
    UpdateOne({'_id': 4}, {'$inc': {'j': 1}}, upsert=True),
    ReplaceOne({'j': 1}, {'j': 2})])
pprint(result.bulk_api_result)

{'nInserted': 3,
 'nMatched': 2,
 'nModified': 2,
 'nRemoved': 4,
 'nUpserted': 1,
 'upserted': [{'_id': 4, 'index': 5}],
 'writeConcernErrors': [],
 'writeErrors': []}


발생하는 첫 번째 쓰기 실패(예 : 중복 키 오류)는 나머지 작업을 중단하고 PyMongo는 BulkWriteError를 발생시킵니다. 예외 인스턴스의 세부 정보 속성은 실패가 발생하기 전까지의 실행 결과와 실패를 유발한 작업을 포함하여 실패에 대한 세부 정보를 제공합니다.

In [14]:
from pymongo import InsertOne, DeleteOne, ReplaceOne
from pymongo.errors import BulkWriteError
requests = [
    ReplaceOne({'j': 2}, {'i': 5}),
    InsertOne({'_id': 4}), # Violates the unique key constraint on _id.
    DeleteOne({'i': 5})]
try:
    db.bulk.bulk_write(requests)
except BulkWriteError as bwe:
    pprint(bwe.details)

{'nInserted': 0,
 'nMatched': 0,
 'nModified': 0,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': [{'code': 11000,
                  'errmsg': 'E11000 duplicate key error collection: '
                            'bulk_example.bulk index: _id_ dup key: { _id: 4 }',
                  'index': 1,
                  'keyPattern': {'_id': 1},
                  'keyValue': {'_id': 4},
                  'op': {'_id': 4}}]}


정렬되지 않은 Bulk Write 작업

정렬되지 않은 Bulk Write 작업은 일괄 처리되어 병렬로 실행될 수 있는 임의의 순서로 서버에 전송됩니다. 모든 작업이 시도 된 후에 발생하는 모든 오류가 보고 됩니다.

다음 예에서는 _id에 대한 고유 제약 조건으로 인해 첫 번째 및 세 번째 작업이 실패합니다. 순서가 지정되지 않은 실행을 수행하기 때문에 두 번째 및 네 번째 작업이 성공합니다.

In [15]:
requests = [
    InsertOne({'_id': 1}),
    DeleteOne({'_id': 2}),
    InsertOne({'_id': 3}),
    ReplaceOne({'_id': 4}, {'i': 1})]
try:
    db.bulk.bulk_write(requests, ordered=False)
except BulkWriteError as bwe:
    pprint(bwe.details)

{'nInserted': 0,
 'nMatched': 1,
 'nModified': 1,
 'nRemoved': 1,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': [{'code': 11000,
                  'errmsg': 'E11000 duplicate key error collection: '
                            'bulk_example.bulk index: _id_ dup key: { _id: 1 }',
                  'index': 0,
                  'keyPattern': {'_id': 1},
                  'keyValue': {'_id': 1},
                  'op': {'_id': 1}},
                 {'code': 11000,
                  'errmsg': 'E11000 duplicate key error collection: '
                            'bulk_example.bulk index: _id_ dup key: { _id: 3 }',
                  'index': 2,
                  'keyPattern': {'_id': 1},
                  'keyValue': {'_id': 3},
                  'op': {'_id': 3}}]}


Write Concern

대량 작업은 실행 되는 컬렉션의 write_concern으로 실행 됩니다. 쓰기 문제 오류 (예 : wtimeout)는 실행 순서에 관계없이 모든 작업이 시도 된 후에 보고 됩니다.

In [18]:
from pymongo import WriteConcern
coll = db.get_collection(
     'test', write_concern=WriteConcern(w=3, wtimeout=1))
try:
    coll.bulk_write([InsertOne({'a': i}) for i in range(4)])
except BulkWriteError as bwe:
    pprint(bwe.details)

OperationFailure: cannot use 'w' > 1 when a host is not replicated, full error: {'ok': 0.0, 'errmsg': "cannot use 'w' > 1 when a host is not replicated", 'code': 2, 'codeName': 'BadValue'}